# 8-puzzle Solver
#### Welcome To  My First 8 Puzzle Solver
<br>
The 8 puzzle consists of 3 by 3 square tiles with one tile missing. Each number represents a tile on the puzzle.  The missing tile is represented with 0 in this solver. The goal is to solve the puzzle by getting the initial state to match the target state by shuffling the tiles around. This solver uses an A* Search algorithm optimization. The initial state is the arrangement of the tiles on the puzzle before it's been tried to be solved. In this state no search algorithm has been implemented. The target state is the state the puzzle should be in when solved.

This algorithm uses two heuristic cost functions:
<br>
heauristicOne(N) is the sum of the (Manhattan) distances of every tile to its goal position (doesn't include empty tile)

heauristicTwo(N) is the number of misplaced tiles (doesn't include empty tile).



In [1]:
import numpy as np
import time
import sys

In [2]:
# define Python user-defined exceptions
class Error(Exception):
    """Base class for other exceptions"""
    pass


class NotInteger(Error):
    """Raised when the input value is not an integer"""
    pass


class AlreadyInputted(Error):
    """Raised when the input value is already n the list"""
    pass


class ValueOutBounds(Error):
    """Raised when the input value is not within the range of 0-9"""
    pass

In [3]:

class Node():
    def __init__(self,state,parentNode,slide,depth,transitionCost,pathCost,heuristicCost):
        self.state = state 
        self.parentNode = parentNode # parentNode node
        self.slide = slide # move up, left, down, right
        self.depth = depth # depth of the node in the tree
        self.transitionCost = transitionCost # g(n), the cost to take the step
        self.pathCost = pathCost # accumulated g(n), the cost to reach the current node
        self.heuristicCost = heuristicCost # h(n), cost to reach goal state from the current node
        
        # children node
        self.slideUp = None 
        self.slideLeft = None
        self.slideDown = None
        self.slideRight = None
    
    # see if moving down is valid
    def trySlideDown(self):
        # index of the empty tile
        zeroTile=[i[0] for i in np.where(self.state==0)] 
        if zeroTile[0] == 0:
            return False
        else:
            upVal = self.state[zeroTile[0]-1,zeroTile[1]] # value of the upper tile
            newState = self.state.copy()
            newState[zeroTile[0],zeroTile[1]] = upVal
            newState[zeroTile[0]-1,zeroTile[1]] = 0
            return newState,upVal
        
    # see if moving right is valid
    def trySlideRight(self):
        zeroTile=[i[0] for i in np.where(self.state==0)] 
        if zeroTile[1] == 0:
            return False
        else:
            leftVal = self.state[zeroTile[0],zeroTile[1]-1] # value of the left tile
            newState = self.state.copy()
            newState[zeroTile[0],zeroTile[1]] = leftVal
            newState[zeroTile[0],zeroTile[1]-1] = 0
            return newState,leftVal
        
    # see if moving up is valid
    def trySlideUp(self):
        zeroTile=[i[0] for i in np.where(self.state==0)] 
        if zeroTile[0] == 2:
            return False
        else:
            lowerVal = self.state[zeroTile[0]+1,zeroTile[1]] # value of the lower tile
            newState = self.state.copy()
            newState[zeroTile[0],zeroTile[1]] = lowerVal
            newState[zeroTile[0]+1,zeroTile[1]] = 0
            return newState,lowerVal
        
    # see if moving left is valid
    def trySlideLeft(self):
        zeroTile=[i[0] for i in np.where(self.state==0)] 
        if zeroTile[1] == 2:
            return False
        else:
            rightVal = self.state[zeroTile[0],zeroTile[1]+1] # value of the right tile
            newState = self.state.copy()
            newState[zeroTile[0],zeroTile[1]] = rightVal
            newState[zeroTile[0],zeroTile[1]+1] = 0
            return newState,rightVal
    
    # return user specified heuristic cost
    def getHCost(self,newState,targetState,heuristicFunct,pathCost,depth):
        if heuristicFunct == 'num_tile_misplaced':
            return self.hMisplacedCost(newState,targetState)
        elif heuristicFunct == 'manhattan':
            return self.hManhattanCost(newState,targetState) - pathCost + depth
    
    # return heuristic cost: number of misplaced tiles
    def hMisplacedCost(self,newState,targetState):
        cost = np.sum(newState != targetState)-1 # minus 1 to exclude the empty tile
        if cost > 0:
            return cost
        else:
            return 0 # when all tiles matches
    
    # return heuristic cost: sum of Manhattan distance to reach the goal state
    def hManhattanCost(self,newState,targetState):
        current = newState
        
        coord = [(0,0), (0,1), (0,2), (1,0), (1,1), (1,2), (2,0), (2,1), (2,2)]
        targetState = np.reshape(targetState, 9)
        targetPosDic = dict(zip(targetState, coord))

        manhattanSum = 0
        for i in range(3):
            for j in range(3):
                if current[i,j] != 0:
                    manhattanSum += sum(abs(a-b) for a,b in zip((i,j), targetPosDic[current[i,j]]))
        return manhattanSum
        
    # once the goal node is found, tracks back to the root node and print out the path
    def printPath(self):
        # create FILO stacks to track the states
        stateTrack = [self.state]
        slideTrack = [self.slide]
        depthTrack = [self.depth]
        transitionCostTrack = [self.transitionCost]
        pathCostTrack = [self.pathCost]
        hCostTrack = [self.heuristicCost]
        
        # add node information as tracking back up the tree
        while self.parentNode:
            self = self.parentNode
            stateTrack.append(self.state)
            slideTrack.append(self.slide)
            depthTrack.append(self.depth)
            transitionCostTrack.append(self.transitionCost)
            pathCostTrack.append(self.pathCost)
            hCostTrack.append(self.heuristicCost)

        # print out the path
        stepCounter = 0
        while stateTrack:
            print ('step',stepCounter)
            print (stateTrack.pop())
            print ('slide =',slideTrack.pop(),', depth =',str(depthTrack.pop()),\
            ', step cost =',str(transitionCostTrack.pop()),', total cost =',\
            str(pathCostTrack.pop() + hCostTrack.pop()),'\n')
            
            stepCounter += 1
            
        print ('No of Nodes Expanded:', str(stepCounter))
            
            
    # search based on path cost + heuristic cost
    def aStarSearch(self,targetState,heuristicFunct):
        start = time.time()
        
        queue = [(self,0)] # queue of (found but unvisited nodes, path cost+heuristic cost), ordered by the second element
        queueNumNodesPopped = 0 # number of nodes popped off the queue, measuring time performance
        queueMaxLen = 1 # max number of nodes in the queue, measuring space performance
        
        depthQueue = [(0,0)] # queue of node depth, (depth, pathCost+heuristic cost)
        pathCostQueue = [(0,0)] # queue for path cost, (pathCost, pathCost+heuristic cost)
        visited = set([]) # record visited states
        
        while queue:
            # sort queue based on pathCost+heuristic cost, in ascending order
            queue = sorted(queue, key=lambda x: x[1])
            depthQueue = sorted(depthQueue, key=lambda x: x[1])
            pathCostQueue = sorted(pathCostQueue, key=lambda x: x[1])
            
            # update maximum length of the queue
            if len(queue) > queueMaxLen:
                queueMaxLen = len(queue)
                
            currentNode = queue.pop(0)[0] # select and remove the first node in the queue
            
            queueNumNodesPopped += 1 
            currentDepth = depthQueue.pop(0)[0] # select and remove the depth for current node
            currentPathCost = pathCostQueue.pop(0)[0] # # select and remove the path cost for reaching current node
            visited.add(tuple(currentNode.state.reshape(1,9)[0])) # avoid repeated state, which is represented as a tuple
            
            # when the goal state is found, track back to the root node and print out the path
            if np.array_equal(currentNode.state,targetState):
                currentNode.printPath()
                
                print ('No of Nodes Generated:',str(queueNumNodesPopped))
                print ('Max No of Nodes in Queue:', str(queueMaxLen))
                print ('Time Complexity: %0.2fs' % (time.time() - start))
                return True
            
            elif ((time.time()) - start) > 180:
                break
                return False
            

            else:     
                # see if moving upper tile down is a valid move
                if currentNode.trySlideDown():
                    newState,upVal = currentNode.trySlideDown()
                    # check if the resulting node is already visited
                    if tuple(newState.reshape(1,9)[0]) not in visited:
                        pathCost = currentPathCost + upVal
                        depth = currentDepth + 1
                        # get heuristic cost
                        hCost = self.getHCost(newState,targetState,heuristicFunct,pathCost,depth)
                        # create a new child node
                        totalCost = pathCost + hCost
                        currentNode.slideDown = Node(state = newState,parentNode = currentNode,slide = 'down',depth = depth,\
                                              transitionCost = upVal,pathCost = pathCost,heuristicCost = hCost)
                        queue.append((currentNode.slideDown, totalCost))
                        depthQueue.append((depth, totalCost))
                        pathCostQueue.append((pathCost, totalCost))
                    
                # see if moving left tile to the right is a valid move
                if currentNode.trySlideRight():
                    newState,leftVal = currentNode.trySlideRight()
                    # check if the resulting node is already visited
                    if tuple(newState.reshape(1,9)[0]) not in visited:
                        pathCost = currentPathCost + leftVal
                        depth = currentDepth + 1
                        # get heuristic cost
                        hCost = self.getHCost(newState,targetState,heuristicFunct,pathCost,depth)
                        # create a new child node
                        totalCost = pathCost + hCost
                        currentNode.slideRight = Node(state = newState,parentNode = currentNode,slide = 'right',depth = depth,\
                                              transitionCost = leftVal,pathCost = pathCost,heuristicCost = hCost)
                        queue.append((currentNode.slideRight, totalCost))
                        depthQueue.append((depth, totalCost))
                        pathCostQueue.append((pathCost, totalCost))
                    
                # see if moving lower tile up is a valid move
                if currentNode.trySlideUp():
                    newState,lowerVal = currentNode.trySlideUp()
                    # check if the resulting node is already visited
                    if tuple(newState.reshape(1,9)[0]) not in visited:
                        pathCost = currentPathCost + lowerVal
                        depth = currentDepth + 1
                        # get heuristic cost
                        hCost = self.getHCost(newState,targetState,heuristicFunct,pathCost,depth)
                        # create a new child node
                        totalCost = pathCost + hCost
                        currentNode.slideUp = Node(state = newState,parentNode = currentNode,slide = 'up',depth = depth,\
                                              transitionCost = lowerVal,pathCost = pathCost,heuristicCost = hCost)
                        queue.append((currentNode.slideUp, totalCost))
                        depthQueue.append((depth, totalCost))
                        pathCostQueue.append((pathCost, totalCost))

                # see if moving right tile to the left is a valid move
                if currentNode.trySlideLeft():
                    newState,rightVal = currentNode.trySlideLeft()
                    # check if the resulting node is already visited
                    if tuple(newState.reshape(1,9)[0]) not in visited:
                        pathCost = currentPathCost + rightVal
                        depth = currentDepth + 1
                        # get heuristic cost
                        hCost = self.getHCost(newState,targetState,heuristicFunct,pathCost,depth)
                        # create a new child node
                        totalCost = pathCost+hCost
                        currentNode.slideLeft = Node(state=newState,parentNode=currentNode,slide='left',depth=depth,\
                                              transitionCost=rightVal,pathCost=pathCost,heuristicCost=hCost)
                        queue.append((currentNode.slideLeft, totalCost))
                        depthQueue.append((depth, totalCost))
                        pathCostQueue.append((pathCost, totalCost))




### CASE 1

In [ ]:
print("\nDefine Target State")
print("All inputs should be an integer value between 0-9\n")

#create list (target)
targetState = []
# Puzzle Size
puzzleSize = 9

while (len(targetState)) < puzzleSize:
    
    try:
        inputTileNumber = int(input("Enter a tile number: "))
        if not isinstance(inputTileNumber, int) :
            raise ValueError
        elif (targetState.count(inputTileNumber)) > 0:
            raise AlreadyInputted
        elif (inputTileNumber in range(0, 9)) == False:
            raise ValueOutBounds
        
        targetState.append(inputTileNumber)
        continue
    except AlreadyInputted:
        print("This value is already has been entered to the list, try again!")
        print()
    except ValueError:
        print("This value is not an integer, try again!")
        print()
    except ValueOutBounds:
        print("This value is not an integer between 0-9, try again!")
        print()
    except NotInteger:
        print("This value is not an integer, try again!")
        print()


print("\nGreat! Target State Input Completed.")
print("\nThe target state is: ")
targetState = np.array(targetState).reshape(3,3)
print (targetState)



print("\nDefine Initial State")
print("Input each number on at a time. All inputs should be an integer value between 0-9")

#create list (initial)
initState = []

# Puzzle Size
puzzleSize = 9

while (len(initState)) < puzzleSize:
    
    try:
        inputTileNumber = int(input("Enter a tile number: "))
        if not isinstance(inputTileNumber, int) :
            raise NotInteger
        elif (initState.count(inputTileNumber)) > 0 :
            raise AlreadyInputted
        elif (inputTileNumber in range(0, 9)) == False :
            raise ValueOutBounds
        
        initState.append(inputTileNumber)
        continue
    except AlreadyInputted:
        print("This value is already has been entered to the list, try again!")
        print()
    except ValueError:
        print("This value is is not an integer, try again!")
        print()
    except ValueOutBounds:
        print("This value is not an integer between 0-9, try again!")
        print()
    except NotInteger:
        print("This value is not an integer, try again!")
        print()


print("\nGreat! Initial State Input Completed.")
print("\nThe initial state is: ")
initState = np.array(initState).reshape(3,3)
print (initState)


#n = int(input("\n\nSelect the heuristics to use(enter 1 or 2).\n1. Manhattan distance \n2. Misplaced tiles\n"))
#print()

#if(n == 2): 
#    rootNode = Node(state=initState,parentNode=None,slide=None,depth=0,transitionCost=0,pathCost=0,heuristicCost=0)
#    # A*1 search based on path cost+heuristic cost, using priority queue
#    print('\nNumber of Misplaced Tiles (h1)')
#    rootNode.aStarSearch(targetState,heuristicFunct = 'num_tile_misplaced')



#if(n == 1):
#    rootNode = Node(state=initState,parentNode=None,slide=None,depth=0,transitionCost=0,pathCost=0,heuristicCost=0)
#    # A*2 search based on path cost+heuristic cost, using priority queue
#    print('\nSum of Distances of the Tile from their Target Position (h2)')
#    rootNode.aStarSearch(targetState,heuristicFunct = 'manhattan')


print()
rootNode = Node(state=initState,parentNode=None,slide=None,depth=0,transitionCost=0,pathCost=0,heuristicCost=0)
# A*1 search based on path cost+heuristic cost, using priority queue
print('\n/---- Number of Misplaced Tiles (h1) -------/')
if (rootNode.aStarSearch(targetState,heuristicFunct = 'num_tile_misplaced')) == True:
    print ('Puzzle is Solvable using h1')
    
else: 
    print ("Puzzle search is unsolvable using h1")
    #continue

# A*2 search based on path cost+heuristic cost, using priority queue
print('\n/---- Sum of Distances of the Tile from their Target Position (h2) -------/')
if (rootNode.aStarSearch(targetState,heuristicFunct = 'manhattan')) == True:
    print ('Puzzle is Solvable using h2')
    
else: 
    print ("Puzzle search is unsolvable using h2")
    quit()



Define Target State
All inputs should be an integer value between 0-9

Enter a tile number: 2
Enter a tile number: 3
Enter a tile number: 5
Enter a tile number: 6
Enter a tile number: 8
Enter a tile number: 9
This value is not an integer between 0-9, try again!

Enter a tile number: 0
Enter a tile number: 1
Enter a tile number: 4
Enter a tile number: 7

Great! Target State Input Completed.

The target state is: 
[[2 3 5]
 [6 8 0]
 [1 4 7]]

Define Initial State
Input each number on at a time. All inputs should be an integer value between 0-9
Enter a tile number: 0
Enter a tile number: 1
Enter a tile number: 2
Enter a tile number: 3
Enter a tile number: 4
Enter a tile number: 5
Enter a tile number: 6
Enter a tile number: 7
Enter a tile number: 8

Great! Initial State Input Completed.

The initial state is: 
[[0 1 2]
 [3 4 5]
 [6 7 8]]


/---- Number of Misplaced Tiles (h1) -------/
Puzzle search is unsolvable using h1

/---- Sum of Distances of the Tile from their Target Position (h2) 

### CASE 2

In [ ]:
print("\nDefine Target State")
print("All inputs should be an integer value between 0-9\n")

#create list (target)
targetState = []
# Puzzle Size
puzzleSize = 9

while (len(targetState)) < puzzleSize:
    
    try:
        inputTileNumber = int(input("Enter a tile number: "))
        if not isinstance(inputTileNumber, int) :
            raise ValueError
        elif (targetState.count(inputTileNumber)) > 0:
            raise AlreadyInputted
        elif (inputTileNumber in range(0, 9)) == False:
            raise ValueOutBounds
        
        targetState.append(inputTileNumber)
        continue
    except AlreadyInputted:
        print("This value is already has been entered to the list, try again!")
        print()
    except ValueError:
        print("This value is not an integer, try again!")
        print()
    except ValueOutBounds:
        print("This value is not an integer between 0-9, try again!")
        print()
    except NotInteger:
        print("This value is not an integer, try again!")
        print()


print("\nGreat! Target State Input Completed.")
print("\nThe target state is: ")
targetState = np.array(targetState).reshape(3,3)
print (targetState)



print("\nDefine Initial State")
print("Input each number on at a time. All inputs should be an integer value between 0-9")

#create list (initial)
initState = []

# Puzzle Size
puzzleSize = 9

while (len(initState)) < puzzleSize:
    
    try:
        inputTileNumber = int(input("Enter a tile number: "))
        if not isinstance(inputTileNumber, int) :
            raise NotInteger
        elif (initState.count(inputTileNumber)) > 0 :
            raise AlreadyInputted
        elif (inputTileNumber in range(0, 9)) == False :
            raise ValueOutBounds
        
        initState.append(inputTileNumber)
        continue
    except AlreadyInputted:
        print("This value is already has been entered to the list, try again!")
        print()
    except ValueError:
        print("This value is is not an integer, try again!")
        print()
    except ValueOutBounds:
        print("This value is not an integer between 0-9, try again!")
        print()
    except NotInteger:
        print("This value is not an integer, try again!")
        print()


print("\nGreat! Initial State Input Completed.")
print("\nThe initial state is: ")
initState = np.array(initState).reshape(3,3)
print (initState)


#n = int(input("\n\nSelect the heuristics to use(enter 1 or 2).\n1. Manhattan distance \n2. Misplaced tiles\n"))
#print()

#if(n == 2): 
#    rootNode = Node(state=initState,parentNode=None,slide=None,depth=0,transitionCost=0,pathCost=0,heuristicCost=0)
#    # A*1 search based on path cost+heuristic cost, using priority queue
#    print('\nNumber of Misplaced Tiles (h1)')
#    rootNode.aStarSearch(targetState,heuristicFunct = 'num_tile_misplaced')



#if(n == 1):
#    rootNode = Node(state=initState,parentNode=None,slide=None,depth=0,transitionCost=0,pathCost=0,heuristicCost=0)
#    # A*2 search based on path cost+heuristic cost, using priority queue
#    print('\nSum of Distances of the Tile from their Target Position (h2)')
#    rootNode.aStarSearch(targetState,heuristicFunct = 'manhattan')


print()
rootNode = Node(state=initState,parentNode=None,slide=None,depth=0,transitionCost=0,pathCost=0,heuristicCost=0)
# A*1 search based on path cost+heuristic cost, using priority queue
print('\n/---- Number of Misplaced Tiles (h1) -------/')
if (rootNode.aStarSearch(targetState,heuristicFunct = 'num_tile_misplaced')) == True:
    print ('Puzzle is Solvable using h1')
    
else: 
    print ("Puzzle search is unsolvable using h1")
    #continue

# A*2 search based on path cost+heuristic cost, using priority queue
print('\n/---- Sum of Distances of the Tile from their Target Position (h2) -------/')
if (rootNode.aStarSearch(targetState,heuristicFunct = 'manhattan')) == True:
    print ('Puzzle is Solvable using h2')
    
else: 
    print ("Puzzle search is unsolvable using h2")
    quit()



### CASE 3

In [ ]:
print("\nDefine Target State")
print("All inputs should be an integer value between 0-9\n")

#create list (target)
targetState = []
# Puzzle Size
puzzleSize = 9

while (len(targetState)) < puzzleSize:
    
    try:
        inputTileNumber = int(input("Enter a tile number: "))
        if not isinstance(inputTileNumber, int) :
            raise ValueError
        elif (targetState.count(inputTileNumber)) > 0:
            raise AlreadyInputted
        elif (inputTileNumber in range(0, 9)) == False:
            raise ValueOutBounds
        
        targetState.append(inputTileNumber)
        continue
    except AlreadyInputted:
        print("This value is already has been entered to the list, try again!")
        print()
    except ValueError:
        print("This value is not an integer, try again!")
        print()
    except ValueOutBounds:
        print("This value is not an integer between 0-9, try again!")
        print()
    except NotInteger:
        print("This value is not an integer, try again!")
        print()


print("\nGreat! Target State Input Completed.")
print("\nThe target state is: ")
targetState = np.array(targetState).reshape(3,3)
print (targetState)



print("\nDefine Initial State")
print("Input each number on at a time. All inputs should be an integer value between 0-9")

#create list (initial)
initState = []

# Puzzle Size
puzzleSize = 9

while (len(initState)) < puzzleSize:
    
    try:
        inputTileNumber = int(input("Enter a tile number: "))
        if not isinstance(inputTileNumber, int) :
            raise NotInteger
        elif (initState.count(inputTileNumber)) > 0 :
            raise AlreadyInputted
        elif (inputTileNumber in range(0, 9)) == False :
            raise ValueOutBounds
        
        initState.append(inputTileNumber)
        continue
    except AlreadyInputted:
        print("This value is already has been entered to the list, try again!")
        print()
    except ValueError:
        print("This value is is not an integer, try again!")
        print()
    except ValueOutBounds:
        print("This value is not an integer between 0-9, try again!")
        print()
    except NotInteger:
        print("This value is not an integer, try again!")
        print()


print("\nGreat! Initial State Input Completed.")
print("\nThe initial state is: ")
initState = np.array(initState).reshape(3,3)
print (initState)


#n = int(input("\n\nSelect the heuristics to use(enter 1 or 2).\n1. Manhattan distance \n2. Misplaced tiles\n"))
#print()

#if(n == 2): 
#    rootNode = Node(state=initState,parentNode=None,slide=None,depth=0,transitionCost=0,pathCost=0,heuristicCost=0)
#    # A*1 search based on path cost+heuristic cost, using priority queue
#    print('\nNumber of Misplaced Tiles (h1)')
#    rootNode.aStarSearch(targetState,heuristicFunct = 'num_tile_misplaced')



#if(n == 1):
#    rootNode = Node(state=initState,parentNode=None,slide=None,depth=0,transitionCost=0,pathCost=0,heuristicCost=0)
#    # A*2 search based on path cost+heuristic cost, using priority queue
#    print('\nSum of Distances of the Tile from their Target Position (h2)')
#    rootNode.aStarSearch(targetState,heuristicFunct = 'manhattan')


print()
rootNode = Node(state=initState,parentNode=None,slide=None,depth=0,transitionCost=0,pathCost=0,heuristicCost=0)
# A*1 search based on path cost+heuristic cost, using priority queue
print('\n/---- Number of Misplaced Tiles (h1) -------/')
if (rootNode.aStarSearch(targetState,heuristicFunct = 'num_tile_misplaced')) == True:
    print ('Puzzle is Solvable using h1')
    
else: 
    print ("Puzzle search is unsolvable using h1")
    #continue

# A*2 search based on path cost+heuristic cost, using priority queue
print('\n/---- Sum of Distances of the Tile from their Target Position (h2) -------/')
if (rootNode.aStarSearch(targetState,heuristicFunct = 'manhattan')) == True:
    print ('Puzzle is Solvable using h2')
    
else: 
    print ("Puzzle search is unsolvable using h2")
    quit()



### CASE 4

In [ ]:
print("\nDefine Target State")
print("All inputs should be an integer value between 0-9\n")

#create list (target)
targetState = []
# Puzzle Size
puzzleSize = 9

while (len(targetState)) < puzzleSize:
    
    try:
        inputTileNumber = int(input("Enter a tile number: "))
        if not isinstance(inputTileNumber, int) :
            raise ValueError
        elif (targetState.count(inputTileNumber)) > 0:
            raise AlreadyInputted
        elif (inputTileNumber in range(0, 9)) == False:
            raise ValueOutBounds
        
        targetState.append(inputTileNumber)
        continue
    except AlreadyInputted:
        print("This value is already has been entered to the list, try again!")
        print()
    except ValueError:
        print("This value is not an integer, try again!")
        print()
    except ValueOutBounds:
        print("This value is not an integer between 0-9, try again!")
        print()
    except NotInteger:
        print("This value is not an integer, try again!")
        print()


print("\nGreat! Target State Input Completed.")
print("\nThe target state is: ")
targetState = np.array(targetState).reshape(3,3)
print (targetState)



print("\nDefine Initial State")
print("Input each number on at a time. All inputs should be an integer value between 0-9")

#create list (initial)
initState = []

# Puzzle Size
puzzleSize = 9

while (len(initState)) < puzzleSize:
    
    try:
        inputTileNumber = int(input("Enter a tile number: "))
        if not isinstance(inputTileNumber, int) :
            raise NotInteger
        elif (initState.count(inputTileNumber)) > 0 :
            raise AlreadyInputted
        elif (inputTileNumber in range(0, 9)) == False :
            raise ValueOutBounds
        
        initState.append(inputTileNumber)
        continue
    except AlreadyInputted:
        print("This value is already has been entered to the list, try again!")
        print()
    except ValueError:
        print("This value is is not an integer, try again!")
        print()
    except ValueOutBounds:
        print("This value is not an integer between 0-9, try again!")
        print()
    except NotInteger:
        print("This value is not an integer, try again!")
        print()


print("\nGreat! Initial State Input Completed.")
print("\nThe initial state is: ")
initState = np.array(initState).reshape(3,3)
print (initState)


#n = int(input("\n\nSelect the heuristics to use(enter 1 or 2).\n1. Manhattan distance \n2. Misplaced tiles\n"))
#print()

#if(n == 2): 
#    rootNode = Node(state=initState,parentNode=None,slide=None,depth=0,transitionCost=0,pathCost=0,heuristicCost=0)
#    # A*1 search based on path cost+heuristic cost, using priority queue
#    print('\nNumber of Misplaced Tiles (h1)')
#    rootNode.aStarSearch(targetState,heuristicFunct = 'num_tile_misplaced')



#if(n == 1):
#    rootNode = Node(state=initState,parentNode=None,slide=None,depth=0,transitionCost=0,pathCost=0,heuristicCost=0)
#    # A*2 search based on path cost+heuristic cost, using priority queue
#    print('\nSum of Distances of the Tile from their Target Position (h2)')
#    rootNode.aStarSearch(targetState,heuristicFunct = 'manhattan')


print()
rootNode = Node(state=initState,parentNode=None,slide=None,depth=0,transitionCost=0,pathCost=0,heuristicCost=0)
# A*1 search based on path cost+heuristic cost, using priority queue
print('\n/---- Number of Misplaced Tiles (h1) -------/')
if (rootNode.aStarSearch(targetState,heuristicFunct = 'num_tile_misplaced')) == True:
    print ('Puzzle is Solvable using h1')
    
else: 
    print ("Puzzle search is unsolvable using h1")
    #continue

# A*2 search based on path cost+heuristic cost, using priority queue
print('\n/---- Sum of Distances of the Tile from their Target Position (h2) -------/')
if (rootNode.aStarSearch(targetState,heuristicFunct = 'manhattan')) == True:
    print ('Puzzle is Solvable using h2')
    
else: 
    print ("Puzzle search is unsolvable using h2")
    quit()
    

### CASE 5

In [ ]:
print("\nDefine Target State")
print("All inputs should be an integer value between 0-9\n")

#create list (target)
targetState = []
# Puzzle Size
puzzleSize = 9

while (len(targetState)) < puzzleSize:
    
    try:
        inputTileNumber = int(input("Enter a tile number: "))
        if not isinstance(inputTileNumber, int) :
            raise ValueError
        elif (targetState.count(inputTileNumber)) > 0:
            raise AlreadyInputted
        elif (inputTileNumber in range(0, 9)) == False:
            raise ValueOutBounds
        
        targetState.append(inputTileNumber)
        continue
    except AlreadyInputted:
        print("This value is already has been entered to the list, try again!")
        print()
    except ValueError:
        print("This value is not an integer, try again!")
        print()
    except ValueOutBounds:
        print("This value is not an integer between 0-9, try again!")
        print()
    except NotInteger:
        print("This value is not an integer, try again!")
        print()


print("\nGreat! Target State Input Completed.")
print("\nThe target state is: ")
targetState = np.array(targetState).reshape(3,3)
print (targetState)



print("\nDefine Initial State")
print("Input each number on at a time. All inputs should be an integer value between 0-9")

#create list (initial)
initState = []

# Puzzle Size
puzzleSize = 9

while (len(initState)) < puzzleSize:
    
    try:
        inputTileNumber = int(input("Enter a tile number: "))
        if not isinstance(inputTileNumber, int) :
            raise NotInteger
        elif (initState.count(inputTileNumber)) > 0 :
            raise AlreadyInputted
        elif (inputTileNumber in range(0, 9)) == False :
            raise ValueOutBounds
        
        initState.append(inputTileNumber)
        continue
    except AlreadyInputted:
        print("This value is already has been entered to the list, try again!")
        print()
    except ValueError:
        print("This value is is not an integer, try again!")
        print()
    except ValueOutBounds:
        print("This value is not an integer between 0-9, try again!")
        print()
    except NotInteger:
        print("This value is not an integer, try again!")
        print()


print("\nGreat! Initial State Input Completed.")
print("\nThe initial state is: ")
initState = np.array(initState).reshape(3,3)
print (initState)


#n = int(input("\n\nSelect the heuristics to use(enter 1 or 2).\n1. Manhattan distance \n2. Misplaced tiles\n"))
#print()

#if(n == 2): 
#    rootNode = Node(state=initState,parentNode=None,slide=None,depth=0,transitionCost=0,pathCost=0,heuristicCost=0)
#    # A*1 search based on path cost+heuristic cost, using priority queue
#    print('\nNumber of Misplaced Tiles (h1)')
#    rootNode.aStarSearch(targetState,heuristicFunct = 'num_tile_misplaced')



#if(n == 1):
#    rootNode = Node(state=initState,parentNode=None,slide=None,depth=0,transitionCost=0,pathCost=0,heuristicCost=0)
#    # A*2 search based on path cost+heuristic cost, using priority queue
#    print('\nSum of Distances of the Tile from their Target Position (h2)')
#    rootNode.aStarSearch(targetState,heuristicFunct = 'manhattan')


print()
rootNode = Node(state=initState,parentNode=None,slide=None,depth=0,transitionCost=0,pathCost=0,heuristicCost=0)
# A*1 search based on path cost+heuristic cost, using priority queue
print('\n/---- Number of Misplaced Tiles (h1) -------/')
if (rootNode.aStarSearch(targetState,heuristicFunct = 'num_tile_misplaced')) == True:
    print ('Puzzle is Solvable using h1')
    
else: 
    print ("Puzzle search is unsolvable using h1")
    #continue

# A*2 search based on path cost+heuristic cost, using priority queue
print('\n/---- Sum of Distances of the Tile from their Target Position (h2) -------/')
if (rootNode.aStarSearch(targetState,heuristicFunct = 'manhattan')) == True:
    print ('Puzzle is Solvable using h2')
    
else: 
    print ("Puzzle search is unsolvable using h2")
    quit()


### CASE 6

In [ ]:
print("\nDefine Target State")
print("All inputs should be an integer value between 0-9\n")

#create list (target)
targetState = []
# Puzzle Size
puzzleSize = 9

while (len(targetState)) < puzzleSize:
    
    try:
        inputTileNumber = int(input("Enter a tile number: "))
        if not isinstance(inputTileNumber, int) :
            raise ValueError
        elif (targetState.count(inputTileNumber)) > 0:
            raise AlreadyInputted
        elif (inputTileNumber in range(0, 9)) == False:
            raise ValueOutBounds
        
        targetState.append(inputTileNumber)
        continue
    except AlreadyInputted:
        print("This value is already has been entered to the list, try again!")
        print()
    except ValueError:
        print("This value is not an integer, try again!")
        print()
    except ValueOutBounds:
        print("This value is not an integer between 0-9, try again!")
        print()
    except NotInteger:
        print("This value is not an integer, try again!")
        print()


print("\nGreat! Target State Input Completed.")
print("\nThe target state is: ")
targetState = np.array(targetState).reshape(3,3)
print (targetState)



print("\nDefine Initial State")
print("Input each number on at a time. All inputs should be an integer value between 0-9")

#create list (initial)
initState = []

# Puzzle Size
puzzleSize = 9

while (len(initState)) < puzzleSize:
    
    try:
        inputTileNumber = int(input("Enter a tile number: "))
        if not isinstance(inputTileNumber, int) :
            raise NotInteger
        elif (initState.count(inputTileNumber)) > 0 :
            raise AlreadyInputted
        elif (inputTileNumber in range(0, 9)) == False :
            raise ValueOutBounds
        
        initState.append(inputTileNumber)
        continue
    except AlreadyInputted:
        print("This value is already has been entered to the list, try again!")
        print()
    except ValueError:
        print("This value is is not an integer, try again!")
        print()
    except ValueOutBounds:
        print("This value is not an integer between 0-9, try again!")
        print()
    except NotInteger:
        print("This value is not an integer, try again!")
        print()


print("\nGreat! Initial State Input Completed.")
print("\nThe initial state is: ")
initState = np.array(initState).reshape(3,3)
print (initState)


#n = int(input("\n\nSelect the heuristics to use(enter 1 or 2).\n1. Manhattan distance \n2. Misplaced tiles\n"))
#print()

#if(n == 2): 
#    rootNode = Node(state=initState,parentNode=None,slide=None,depth=0,transitionCost=0,pathCost=0,heuristicCost=0)
#    # A*1 search based on path cost+heuristic cost, using priority queue
#    print('\nNumber of Misplaced Tiles (h1)')
#    rootNode.aStarSearch(targetState,heuristicFunct = 'num_tile_misplaced')



#if(n == 1):
#    rootNode = Node(state=initState,parentNode=None,slide=None,depth=0,transitionCost=0,pathCost=0,heuristicCost=0)
#    # A*2 search based on path cost+heuristic cost, using priority queue
#    print('\nSum of Distances of the Tile from their Target Position (h2)')
#    rootNode.aStarSearch(targetState,heuristicFunct = 'manhattan')


print()
rootNode = Node(state=initState,parentNode=None,slide=None,depth=0,transitionCost=0,pathCost=0,heuristicCost=0)
# A*1 search based on path cost+heuristic cost, using priority queue
print('\n/---- Number of Misplaced Tiles (h1) -------/')
if (rootNode.aStarSearch(targetState,heuristicFunct = 'num_tile_misplaced')) == True:
    print ('Puzzle is Solvable using h1')
    
else: 
    print ("Puzzle search is unsolvable using h1")
    #continue

# A*2 search based on path cost+heuristic cost, using priority queue
print('\n/---- Sum of Distances of the Tile from their Target Position (h2) -------/')
if (rootNode.aStarSearch(targetState,heuristicFunct = 'manhattan')) == True:
    print ('Puzzle is Solvable using h2')
    
else: 
    print ("Puzzle search is unsolvable using h2")
    quit()
